In [1]:
%matplotlib inline
import collections 
import numpy as np
import scipy as sp
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import re
#import regex
#import locale
#import usaddress
#import geograpy
#import nltk
from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
import requests

In [2]:
# global data storage
climbsjson = collections.defaultdict(dict) # double nested dict
usersjson = collections.defaultdict(dict) # double nested dict
ucjson = collections.defaultdict(dict) #double nested dict

In [3]:
# sub-functions and initializations for user/climb_scrape_func-s

def parse_climb_link(href_link):
    return href_link.split('/')[-1], href_link.split('/')[-2]

def parse_user_link(href_link):
    return href_link.split('/')[-1], href_link.split('/')[-3]

aid_re = r'(A[0-5])|(C[0-5])[-,+]?'
boulder_re = r'V\d\d?[-,+]?'
ice_re = r'WI[1-6][+,-]?'
mixed_re = r'M\d\d?[+,-]?'
rock_re = r'5.\d\d?(([a-d]/[a-d])|([a-d]))?[+,-]?'

diff_regexes = {"aid_grade": aid_re, "boulder_grade": boulder_re, 
               "ice_grade": ice_re, "mixed_grade": mixed_re, 
               "rock_grade": rock_re}

user_type_regexes = {"Trad": rock_re, "Sport": rock_re, 
                     "Boulders": boulder_re, "Aid": aid_re, 
                     "Ice": ice_re, "Mixed": mixed_re}

typesdict = {"trad": 0, "alpine": 0, "ice": 0, "sport": 0,
        "boulder": 0, "aid": 0, "mixed": 0, "TR,": 0}

In [4]:
userlink = "http://mountainproject.com/u/jared-lavacque//106325793"

userdict = {"user_id": 0, "user_name": "undef", "location_place": "undef", 
            "location_state": "undef", "age": 0, "gender": "undef", 
            "member_date": 0}

user_id, user_name = parse_user_link(userlink)

userdict["user_id"], userdict["user_name"] = user_id, user_name

In [5]:
user_req=requests.get(userlink)
user_=pq(user_req.text)

In [6]:
user_main = user_('div.personalData')

In [7]:
user_pers = user_main("div:contains('Personal:')")('em')

In [8]:
if re.search('Lives in', user_pers.text()) != None:
        userdict["location_place"] = user_pers.text().split(',')[0].replace('Lives in ',"")
        maybe_state = user_pers.text().split(',')[1]
        if (re.search('(years old)|(male)|(female)', maybe_state) == None):
            userdict["location_state"] = maybe_state

In [9]:
temp_age = re.search(r'(\d\d?) years old', user_pers.text())
if temp_age:
    userdict["age"] = int(temp_age.group(1))

In [10]:
temp_gender = re.search(r'(male)|(female)', user_pers.text(), re.I)
if temp_gender:
    userdict["gender"] = temp_gender.group()

In [11]:
 # the below regretably doesnt work because the selector :contains cannot 
# format strings...

user_type_regexes = {'Trad:': rock_re, "Sport": rock_re, 
                     "Boulders": boulder_re, "Aid": aid_re, 
                     "Ice": ice_re, "Mixed": mixed_re}


user_diffs = user_main('table')

for key, value in user_type_regexes.items():
    if key != "Boulders":
        tdwithkey = user_diffs.find("tr").children("td:contains('{}')".format(key))
        if tdwithkey:
            userdict["climbs_" + str.lower(key)] = 1
            leadhtml = tdwithkey.next().html()
            followhtml = tdwithkey.next().next().html()  
            templeaddiff = re.search("{}".format(value), leadhtml, re.I)
            tempfollowdiff = re.search("{}".format(value), followhtml, re.I)
            if templeaddiff:
                userdict["lead_diff_" + str.lower(key)] = templeaddiff.group() 
            else:
                userdict["lead_diff_" + str.lower(key)] = '0'
            if tempfollowdiff:
                userdict["follow_diff_" + str.lower(key)] = tempfollowdiff.group()
            else:
                userdict["follow_diff_" + str.lower(key)] = '0'
        else:
            userdict["climbs_" + str.lower(key)] = 0

    elif key == "Boulders":
        boulderhtml = user_diffs.find("tr").children("td:contains('{}')".format(key))
        if boulderhtml:
            userdict["climbs_boulder"] = 1
            boulderdiff = boulderhtml.next().html()
            tempboulderdiff = re.search("{}".format(value), boulderdiff)
            if tempboulderdiff:
                userdict["boulder_diff"] = tempboulderdiff.group()
        else:
            userdict["climbs_boulder"] = 0
            


In [18]:
# 6. get date 
user_left_box = user_("td[width='190']")('b')
if user_left_box:
    userdict["joined_date"] = time.strptime(user_left_box.eq(0).text(), 
                                            "%b %d, %Y")

SyntaxError: invalid syntax (<ipython-input-18-53d5e5db969a>, line 5)

In [19]:
starslink = "http://www.mountainproject.com/u/{}?action=contribs&what=SCORE&".format(user_id)
stars_req=requests.get(starslink)
user_stars=pq(stars_req.text)
user_stars("table[width*='100%'][border='0'][class='objectList']").eq(1).find('tr')

[<tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>, <tr>]

In [43]:
tickslink = "http://www.mountainproject.com/u/{}?action=ticks&".format(user_id)
#tickslink = "http://www.mountainproject.com/u/ian-grant//108290010?action=ticks&"
# 7. get ticks
ticks_req=requests.get(tickslink)
ticks_page = pq(ticks_req.text)
num_pages = ticks_page("table#stats").next().next().find('tr').eq(0).text()
if num_pages:
    pages = int(re.search(r'Page \d\d? of (\d\d?)', num_pages).group(1))
    for page in range(1, pages + 1):
        
    
tick_rows = ticks_page("table[width*='100%'][border='0'][class='objectList']").eq(1).find('tr')   



http://www.mountainproject.com/u/106325793?action=ticks&


u'7'

In [30]:
for i in range(2, len(tick_rows)):
    tds = tick_rows.eq(i).find('td')
    climb_link = tds.eq(0).find('a').attr('href')
    climb_id, _ = climb_link.split('/')[-1], climb_link.split('/')[-2]
    date = time.strptime(tds.eq(4)('p').text(), "%b %d, %Y")
    ucjson[climb_id + "_" + user_id]["user_id"] = user_id
    ucjson[climb_id + "_" + user_id]["climb_id"] = climb_id
    ucjson[climb_id + "_" + user_id]["ticked_date"] = date


							<td class="objectListColumnHeader">Name</td>
							<td class="objectListColumnHeader">Rating</td>
							<td class="objectListColumnHeader">Location</td>
							<td class="objectListColumnHeader">Comments</td>
							<td class="objectListColumnHeader">Tick Date</td>
						


In [49]:
if len(usersjson[23131]) == 0:
    print "yay"

yay
